In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/'Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [ ]:
#@markdown #**Clone github & download models**

!git clone https://github.com/liuhui0401/final.git
%cd final

!pip install git+https://github.com/facebookresearch/segment-anything.git
!wget -P ./weights https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

# load arcface
!wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
!wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/iresnet.py

# load landmarks detector
!wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/glintr100.onnx
!wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/scrfd_10g_bnkps.onnx

# load model itself
!wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/sber-swap-v2.0/G_unet_2blocks.pth

# load super res model
!wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/super-res/10_net_G.pth

In [ ]:
#@markdown #**Install required libraries**

!pip install mxnet-cu112
!pip install onnxruntime-gpu==1.12
!pip install insightface==0.2.1
!pip install kornia==0.5.4
!pip install dill

!rm /usr/local/lib/python3.10/dist-packages/insightface/model_zoo/model_zoo.py #change the path to python in case you use a different version
!wget -P /usr/local/lib/python3.10/dist-packages/insightface/model_zoo/ https://github.com/AlexanderGroshev/insightface/releases/download/model_zoo/model_zoo.py #change the path to python in case you use a different version

In [ ]:
#@markdown #**Preparation**

import cv2
import torch
import time
import os
import numpy as np

from utils.inference.image_processing import crop_face, get_final_image, show_images
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement, smooth_landmarks
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions
from insightface.utils import face_align
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#@markdown #**Initialize models**

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

In [ ]:
#@markdown #**Upload video**

target_type = 'video' #@param ["video", "image"]

source_path = ['examples/images/beckham.jpg', 'examples/images/test2.jpg'] #@param {type:"string"}
target_path = ['examples/images/test_target1.jpg'] #@param {type:"string"}
sticker_path = ['examples/images/saihong.jpg'] #@param {type:"string"}
sticker_target_path = ['examples/images/beckham.jpg'] #@param {type:"string"}
path_to_video = 'examples/videos/nggyup.mp4' #@param {type:"string"}
mode = ['cheek'] #@param {type:"string"} ## eyes cheek forehead face hair nose mouth mouth_open left_eye right_eye

OUT_VIDEO_NAME = "examples/results/result.mp4"
crop_size = 224 # don't change this

In [ ]:
from utils.inference.core import model_inference_all, model_inference_all_new
from utils.inference.video_processing import get_final_video_all, generate_video

source_full = []
for source in source_path:
  source_full.append(crop_face(cv2.imread(source), app, crop_size)[0][:, :, ::-1])
target_full = []
for target in target_path:
  target_full.append(crop_face(cv2.imread(target), app, crop_size)[0])
sticker_full = []
for sticker in sticker_path:
  sticker_full.append(cv2.imread(sticker))
sticker_target_full = []
for sticker_target in sticker_target_path:
  sticker_target_full.append(crop_face(cv2.imread(sticker_target), app, crop_size)[0])

model_inference_all_new(full_frames,
            source_full,
            target_full,
            sticker_full,
            sticker_target_full,
            netArc,
            G,
            app,
            set_target = False,
            crop_size=crop_size,
            mode=mode,
            handler=handler,
            OUT_VIDEO_NAME=OUT_VIDEO_NAME,
            fps=fps)


In [ ]:
########   stop here!!!!!!!!!!!!!!